# Assignment

In this assignment, you will be required to **explore, segment, and cluster** the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is **not readily** available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a **Wikipedia page** exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a **pandas dataframe** so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

*Your submission will be a link to your Jupyter Notebook on your Github repository.*

# Part 1: Built a dataframe

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

**Make sure the components are installed**

In [45]:
!pip install beautifulsoup4

In [46]:
!pip install lxml

In [47]:
!pip install requests

**Import the necessary libraries**

In [48]:
import requests # library to handle requests
import lxml.html as lh
import bs4 as bs
import urllib.request

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

Save the URL in a variable

In [49]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Using BeautifulSoup4 i defined a scrape table

In [50]:
# format: scrape_table_bs4 <tableClassName> <expected numberOfColumns>
def scrape_table_bs4(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    # Store data to this temporary dataframe
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df


And defined other one using lxml

In [51]:
# Parsing with xpath
def scrape_table_lxml(XPATH,cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data,columns=headers)
        return raw_df

Test in beautifulSoup

In [52]:
#Test in beautifulSoup
raw_TorontoPostalCodes = scrape_table_bs4("wikitable",3)

print("# Toronto Postal codes stored in data")
print(raw_TorontoPostalCodes.info(verbose=True))

# Toronto Postal codes stored in data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal code   180 non-null    object
 1   Borough       180 non-null    object
 2   Neighborhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB
None


In [53]:
We need to only process the cells that have an assigned borough

SyntaxError: invalid syntax (<ipython-input-53-5adb22a26cfa>, line 1)

In [ ]:
TorontoPostalCodes=raw_TorontoPostalCodes[~raw_TorontoPostalCodes['Borough'].isin(['Not assigned'])]
TorontoPostalCodes

Then sort and reset the index.

In [ ]:
TorontoPostalCodes=TorontoPostalCodes.sort_values(by=['Postal code','Borough','Neighborhood'], ascending=[1,1,1]).reset_index(drop=True)

If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
TorontoPostalCodes.loc[TorontoPostalCodes['Neighborhood'] == 'Not assigned', ['Neighborhood']] = TorontoPostalCodes['Borough']
check_unassigned_post_state_sample = TorontoPostalCodes.loc[TorontoPostalCodes['Borough'] == 'Queen\'s Park']

More than one neighborhood can exist in one postal code area. 
For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [54]:
TorontoPostalCodes = TorontoPostalCodes.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [55]:
TorontoPostalCodes

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


Check the size of the data frame

In [56]:
TorontoPostalCodes.shape

(103, 3)

Save it for the next tasks

In [57]:
TorontoPostalCodes.to_csv('Toronto.TASK_1_df.csv',index=False)